# Cración de dataset con datos de electroencefalografía

In [27]:
import pandas as pd
import sys
import os
import torch
import torchaudio.transforms as T
import numpy as np
import mne
import librosa
import utilit_espectrograms as ue

In [28]:
DATA_DIR        = '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data'
OUTDATA_DIR     = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
OUTMETADATA_DIR = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/'

In [29]:
# crear las carpetas en caso de que no existan

if not os.path.exists(OUTDATA_DIR):
    os.makedirs(OUTDATA_DIR)

if not os.path.exists(OUTMETADATA_DIR):
    os.makedirs(OUTMETADATA_DIR)

In [30]:
# lista con ids de pacientes

RNSIDS = ue.get_subfolders(DATA_DIR)

In [31]:
df = pd.DataFrame(columns=['rns_id', 'data', 'label', 'time'])

ejemplo de un nfile

In [43]:
s      = 0
nepoch = 1
nfile  = 23

In [44]:
data_files  = ue.get_data_files(DATA_DIR, RNSIDS[s], Verbose=False)
annot_files = ue.get_annot_files(DATA_DIR, RNSIDS[s], Verbose=False)

In [47]:
RNSIDS[23]

'PIT-RNS8973'

In [45]:
# signal

events           = ue.get_events(annot_files[nepoch])
X, labels, times = ue.get_epochs_zeropad_all(data_files[nepoch], events)

[nt, nc, ns] = np.shape(X)  

hosp_id, subject_id, PE_id = ue.get_patient_PE(data_files[nepoch], RNSIDS[s])

file   = X[nfile, :, :]
signal = torch.from_numpy(file)
signal = (signal - signal.mean()) / signal.std()

# label
label_time = np.zeros(2)

label_time[0] = labels[nfile]
label_time[1] = times[nfile]

Extracting EDF parameters from /media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS1603/PIT-RNS1603_PE20150909-1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [46]:
label_time.astype(np.float32)

array([ 1.      , 38.261326], dtype=float32)

fin ejemplo de un nfile

In [ ]:
for s in range(len(RNSIDS)):
    
    print('Running subject ' + RNSIDS[s] + ' [s]: ' + str(s))
    data_files  = ue.get_data_files(DATA_DIR, RNSIDS[s], Verbose=False)
    annot_files = ue.get_annot_files(DATA_DIR, RNSIDS[s], Verbose=False)

    for nepoch in range(len(data_files)):

        events           = ue.get_events(annot_files[nepoch])
        X, labels, times = ue.get_epochs_zeropad_all(data_files[nepoch], events)

        [nt, nc, ns] = np.shape(X)        

        hosp_id, subject_id, PE_id = ue.get_patient_PE(data_files[nepoch], RNSIDS[s])

        for nfile in range(nt):
            file   = X[nfile, :, :]
            signal = torch.from_numpy(file)
            signal = (signal - signal.mean()) / signal.std()
            signal = signal.to(torch.float32)

            # label
            label_time = np.zeros(2)
            label_time[0] = labels[nfile]
            label_time[1] = times[nfile]
            label_time.astype(np.float32)

            data      = {'iEEG': signal, 'label_time': label_time}
            file_name = hosp_id + '_' + subject_id + '_' + PE_id + '_E' + str(nfile)

            df_aux = {  
                        'rns_id': hosp_id + '-' + subject_id, 
                        'data'  : file_name,
                        'label' : labels[nfile],
                        'time'  : times[nfile]
                     }
            
            df = pd.concat([df, pd.DataFrame([df_aux])], ignore_index=True)

            np.save(OUTDATA_DIR + file_name, data)

    df.to_csv(OUTMETADATA_DIR + 'allfiles_metadata.csv', index=False)